In [ ]:
#| default_exp datasources.srd
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path

In [ ]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

# Radiodifusão
> Módulo para encapsular a extração e processamento das estações de Radiodifusão Pública

In [ ]:
#| export
import os
from decimal import Decimal
from functools import cached_property

import pandas as pd
from dotenv import find_dotenv, load_dotenv

from extracao.constants import (
    BW_MAP,
    COLUNAS,
    DICT_SRD,
    MONGO_SRD,
    PROJECTION_SRD,
)

from extracao.datasources.mosaico import Mosaico

In [ ]:
#| export
load_dotenv(find_dotenv())

True

In [ ]:
#| hide: true
#| eval:false
__file__ = Path.cwd().parent / 'extracao' / 'datasources.py'

In [ ]:
#| export

MONGO_URI = os.environ.get("MONGO_URI")

In [ ]:
#| export
class SRD(Mosaico):
    """Classe para encapsular a lógica de extração de Radiodifusão"""

    def __init__(self, mongo_uri: str = MONGO_URI) -> None:
        super().__init__(mongo_uri)

    @property
    def stem(self):
        return "srd"

    @property
    def collection(self):
        return "srd"

    @property
    def query(self):
        return MONGO_SRD

    @property
    def projection(self):
        return PROJECTION_SRD

    @property
    def columns(self):
        return COLUNAS

    @property
    def cols_mapping(self):
        return DICT_SRD

    @cached_property
    def extraction(self) -> pd.DataFrame:
        pipeline = [
            # match the documents that satisfy your query
            {"$match": self.query},
            # project the fields that you want to keep
            {"$project": self.projection},
        ]
        df = self._extract(self.collection, pipeline).astype(
            "string"
        )  # Necessária a conversão nesse ponto
        df.loc[df["estacao"] == "[]", "estacao"] = "{}"
        cols = ["srd_planobasico", "estacao", "habilitacao", "Status"]
        for col in cols:
            df = df.join(pd.json_normalize(df[col].apply(eval)))
        df = df.drop(columns=cols)
        df["Log"] = ""
        # Substitui strings vazias e somente com espaços por nulo
        return df.replace(r"^\s*$", pd.NA, regex=True)

    def _format(
        self,
        df: pd.DataFrame,  # DataFrame com o resultantes do banco de dados
    ) -> pd.DataFrame:  # DataFrame formatado
        """Formata, limpa e padroniza os dados provenientes da query no banco"""

        df = df.rename(columns=self.cols_mapping)
        status = df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)
        discarded = df[~status].copy()
        log = """[("Registro", "Status"), 
                ("Processamento", "Registro com Status não considerado para fins de monitoração")]"""
        discarded = self.register_log(discarded, log)
        df = df[
            df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)
        ].reset_index(drop=True)
        df["Frequência"] = (
            df.Frequência.astype("string").str.replace(",", ".").astype("float")
        )
        discarded_with_na = df[df.Frequência.isna()].copy()
        log = """[("Registro", "Frequência"), 
                ("Processamento", "Registro com valor nulo presente")]"""
        discarded_with_na = self.register_log(discarded_with_na, log)
        df = df.dropna(subset="Frequência", ignore_index=True)
        df.loc[df["Num_Serviço"] == "205", "Frequência"] = df.loc[
            df["Num_Serviço"] == "205", "Frequência"
        ].apply(lambda x: float(Decimal(x) / Decimal(1000)))
        df["Frequência"] = df["Frequência"].astype("float")
        df["Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
        df["Fonte"] = "MOSAICO"
        df["Num_Serviço"] = df["Num_Serviço"].fillna("")
        df["Designação_Emissão"] = (
            df.Num_Serviço.astype("string").fillna("").map(BW_MAP)
        )
        df = self.split_designacao(df)
        df["Multiplicidade"] = 1
        self.append2discarded([self.discarded, discarded, discarded_with_na])
        return df.loc[:, self.columns]

In [ ]:
#|export

if __name__ == "__main__":
    import time

    start = time.perf_counter()

    data = SRD()

    data.update()

    print("DATA")

    display(data.df)

    print(150 * "=")

    print("DISCARDED!")

    display(data.discarded[["Frequência", "Entidade", "Log"]])

    print(150 * "=")

    print(data.df.Multiplicidade.sum())

    data.save()

    print(f"Elapsed time: {time.perf_counter() - start} seconds")

DATA


,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade,Log
0,207.0,REDE DE COMUNICACOES ACREANA LTDA,50442889933,NaN,Cruzeiro do Sul,1200203,AC,NaN,NaN,A,248,,6000.0,<NA>,TV-C1,MOSAICO,1,[]
1,539.0,X-MEDIAGROUP S.A.,50410887137,<NA>,Mâncio Lima,1200336,AC,NaN,NaN,C,248,,6000.0,<NA>,TV-C1,MOSAICO,1,[]
2,79.0,TELEVISAO OESTE BAIANO LTDA,06030116240,322647029,Barreiras,2903201,BA,-12.1013888888888333,-44.9936111111110000,A,248,,6000.0,2023-12-31,TV-C4,MOSAICO,1,[]
3,69.0,TELEVISAO SANTA CRUZ LTDA,06020355110,322623553,Itabuna,2914802,BA,-14.7794444444443333,-39.2622222222221666,A,248,,6000.0,2023-12-31,TV-C4,MOSAICO,1,[]
4,177.0,TV CABRALIA LTDA,06020354903,322623537,Itabuna,2914802,BA,-14.7833333333333333,-39.2833333333333333,B,248,,6000.0,2023-12-31,TV-C4,MOSAICO,1,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31098,91.5,EMPRESA BRASIL DE COMUNICACAO S.A. - EBC,50446653047,NaN,Santa Maria da Vitória,2928109,BA,NaN,NaN,B2,230,,256.0,<NA>,FM-C1,MOSAICO,1,[]
31099,87.1,EMPRESA BRASIL DE COMUNICACAO S.A. - EBC,50446654442,NaN,Lavras,3138203,MG,NaN,NaN,B1,230,,256.0,<NA>,FM-C1,MOSAICO,1,[]
31100,87.1,EMPRESA BRASIL DE COMUNICACAO S.A. - EBC,50446653632,NaN,Chapadão do Sul,5002951,MS,NaN,NaN,C,230,,256.0,<NA>,FM-C1,MOSAICO,1,[]
31101,91.1,RADIO CIDADE EUCLIDES DA CUNHA LTDA ME,06020354571,322623499,Euclides da Cunha,2910701,BA,-10.5016666666666666,-39.0338888888888333,A3,230,,256.0,2026-05-05,FM-C4,MOSAICO,1,[]


DISCARDED!


,Frequência,Entidade,Log
0,57.0,FUNDACAO ALDEIA DE COMUNICACAO DO ACRE-FUNDAC,"[]|[(""Registro"", ""Status""), \n ..."
1,69.0,RÁDIO TV DO AMAZONAS LTDA.,"[]|[(""Registro"", ""Status""), \n ..."
2,183.0,SOCIEDADE ACREANA DE COMUNICACAO FRONTEIRA LTDA,"[]|[(""Registro"", ""Status""), \n ..."
3,201.0,RADIO E TELEVISAO NORTE LTDA,"[]|[(""Registro"", ""Status""), \n ..."
4,503.0,REDE UNIAO DE RADIO E TELEVISAO LTDA,"[]|[(""Registro"", ""Status""), \n ..."
...,...,...,...
4522,88.5,SISTEMA DE COMUNICACAO SOL LTDA.,"[]|[(""Registro"", ""Status""), \n ..."
4523,635.0,FUNDACAO NOSSA SENHORA APARECIDA,"[]|[(""Registro"", ""Status""), \n ..."
4524,617.0,PARANA FM LTDA,"[]|[(""Registro"", ""Status""), \n ..."
4525,479.0,PABLO COMUNICACOES LTDA - EPP,"[]|[(""Registro"", ""Status""), \n ..."


31103
Elapsed time: 9.540770499967039 seconds
